# Redes Neuronais + Backpropagation

In [ ]:
from pathlib import Path
from itertools import product
from collections import Counter

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Read Data

In [ ]:
data = pd.read_csv('./data_tp1', header=None)

In [ ]:
features = data.loc[:, 1:]
target = data.loc[:, 0]

In [ ]:
features.shape, target.shape

In [ ]:
Counter(target)

### Model Hiperparameters Options [[docs](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)]

In [ ]:
activation = 'tanh' # Sigmoid

neuron_input = 784
neuron_output = 10

neurons_hidden = [25, 50, 100]
solver = 'sgd'
batch_sizes = [
    10, 50,         # Mini Batch
    1,              # Stochastic Gradient Descent
    target.shape[0] # Gradient Descent
]

learning_rate_inits = [0.5, 1, 10]

In [ ]:
params = list()
for batch_size, learning_rate_init, neuron_hidden in product(batch_sizes, learning_rate_inits, neurons_hidden):
    params.append({
        'activation': activation,
        'hidden_layer_sizes': neuron_hidden,
        'solver': solver,
        'batch_size': batch_size,
        'learning_rate': 'constant',
        'learning_rate_init': learning_rate_init,
        'early_stopping': False,
        'nesterovs_momentum': True,
        'momentum': .9,
        'verbose': False,
        'alpha': .0001,
        'max_iter': 256,  # number of epochs
        'shuffle': True,
        'random_state': None,
        'n_iter_no_change': 10,
        'tol': 1e-4,
    })

In [ ]:
relevant_params = ['batch_size', 'learning_rate_init', 'hidden_layer_sizes']

In [ ]:
len(params)

### Model Instance

In [ ]:
reports = list()
for curr_params in tqdm(params):
    report = dict()
    report['params'] = curr_params

    model = MLPClassifier(**curr_params)
    model.fit(features, target)

    metrics = classification_report(
        y_pred=model.predict(features),
        y_true=target,
        output_dict=True,
        zero_division=0,
    )

    report['metrics'] = metrics
    report['curves'] = {
        'loss_curve': model.loss_curve_,
        'best_loss': model.best_loss_,
        'n_iter': model.n_iter_
    }

    reports.append(report)

### Evaluate Results

In [ ]:
fpath = 'results.csv'
if Path(fpath).exists():
    results = pd.read_csv(fpath)
else:
    results = pd.json_normalize(reports)
    results.to_csv(fpath, index=False)

In [ ]:
params_cols = ['metrics.accuracy'] + [f'params.{param}' for param in relevant_params]
results[params_cols].sort_values('metrics.accuracy', ascending=False).head(10)

In [ ]:
(
    results
    [results['curves.best_loss'] < 10]
    .select_dtypes('number')
    .mean()
)

---